In [ ]:
#|default_exp SebiDataCrawler.end_to_end_crawler

# SEBI DATA CRAWLER

## Any Dependencies?

In [ ]:
#|exports
import logging

import logging
from datetime import datetime
import pytz

def setup_logger(log_file):
    # Set up logging
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    # Set the timezone to Indian Standard Time
    ist = pytz.timezone('Asia/Kolkata')

    # Create a formatter that includes the timestamp in IST
    class ISTFormatter(logging.Formatter):
        def converter(self, timestamp):
            dt = datetime.fromtimestamp(timestamp)
            dt = pytz.utc.localize(dt)
            return dt.astimezone(ist)

    formatter = ISTFormatter('%(asctime)s - %(levelname)s - %(message)s')

    # Create a file handler
    fh = logging.FileHandler(log_file)
    fh.setLevel(logging.INFO)
    fh.setFormatter(formatter)

    # Add the file handler to the logger
    logger.addHandler(fh)

    # Create a console handler
    ch = logging.StreamHandler()
    ch.setLevel(logging.INFO)
    ch.setFormatter(formatter)

    # Add the console handler to the logger
    logger.addHandler(ch)

    return logger



# Global Variables Definitions

In [ ]:
#|exports
#|eval: true
#|code-fold: True

import os

'''
legal_menu_and_sub_menu is dict to store the urls to access "Historical Data" from the legal menu.
It's hard to collect these links, hence need to hardcode.
'''
legal_menu_and_sub_menu = {
            "acts" : "https://www.sebi.gov.in/sebiweb/home/HomeAction.do?doListingLegal=yes&sid=1&ssid=1&smid=0",
            "rules" : "https://www.sebi.gov.in/sebiweb/home/HomeAction.do?doListingLegal=yes&sid=1&ssid=2&smid=0",
            "regulations" : "https://www.sebi.gov.in/sebiweb/home/HomeAction.do?doListingLegal=yes&sid=1&ssid=3&smid=0",
            "general_orders" : "https://www.sebi.gov.in/sebiweb/home/HomeAction.do?doListing=yes&sid=1&ssid=4&smid=0",
            "guidelines" : "https://www.sebi.gov.in/sebiweb/home/HomeAction.do?doListing=yes&sid=1&ssid=5&smid=0",
            "master_circulars" : "https://www.sebi.gov.in/sebiweb/home/HomeAction.do?doListing=yes&sid=1&ssid=6&smid=0",
            "circulars" : "https://www.sebi.gov.in/sebiweb/home/HomeAction.do?doListing=yes&sid=1&ssid=7&smid=0",
            # Doubt ? => How is Circulars are Circular_archive are different?
            "circulars_archive" : "https://www.sebi.gov.in/sebiweb/home/HomeAction.do?doListingCirArchive=yes&sid=1&ssid=7&smid=0",
            "gazette_notification" : "https://www.sebi.gov.in/sebiweb/home/HomeAction.do?doListing=yes&sid=1&ssid=82&smid=0",
            "online_application_portal" : "https://www.sebi.gov.in/sebiweb/home/HomeAction.do?doListing=yes&sid=1&ssid=91&smid=0",
            "guidance_notes" : "https://www.sebi.gov.in/sebiweb/home/HomeAction.do?doListing=yes&sid=1&ssid=85&smid=0",
        }


# '''DB CONSTANTS'''
import os

datanase_name = "crawled_data.sqlite3"
database_path = os.path.join("..", "databases", datanase_name)
DB_NAME = database_path

TABLE_SEBI_RECORDS = "sebi_records"
# '''Paths of Folders and CSV files''' 
home_page_url = "https://www.sebi.gov.in"

regChat_folder = ""
extraction_folder_name = "SEBI_Extracted_Data"

base_folder_path = os.path.join(regChat_folder,extraction_folder_name)

SEBI_data_extraction_base_folder = os.path.join(regChat_folder,extraction_folder_name)

urls_of_sebi_menu_csv_path = os.path.join(regChat_folder,"urls_of_menus_of_sebi.csv")


# Scrape Menus and Folder Creator

1. Use menu_of_sebi.js file from SEBI to collect initial set of links. => save them into csv file
2. Using these menus and submenus collected to create hierachy of folders
3. 

In [ ]:
#|exports
#|eval: true
#|code-fold: True

import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from urllib.parse import urljoin

class SEBIMenuCollector:
    
        def dictify(self, ul_element):
            '''
            Recursively converts nested HTML <ul> elements into a multi-level dictionary.

            Args:
                ul_element (bs4.element.Tag): BeautifulSoup Tag object representing the <ul> element.

            Returns:
                dict: A multi-level dictionary representing the menu structure.

            Use? => This method is used to create a nested folder structure and 
                    maintain hierarchy for storing menus and submenus.

            '''
            result = {}
            # Iterate over each <li> element within the <ul> element
            for li_element in ul_element.find_all("li", recursive=False):
                # Extract the text content of the <li> element to use as the dictionary key
                key = next(li_element.stripped_strings)
                url = ""
                # Check if the <li> element contains an <a> tag (link)
                a_tags = li_element.find_all('a')
                if len(a_tags) == 1:
                    # If only one <a> tag exists, extract the URL
                    url = a_tags[0]['href']
                    
                # Check if the <li> element contains a nested <ul> element (submenu)
                ul_sub_element = li_element.find("ul")
                if ul_sub_element:
                    # Recursively call dictify() to handle nested menus
                    result[key] = self.dictify(ul_sub_element)
                else:
                    # If no submenu exists, check and normalize the URL
                    if not url.startswith("http"):
                        base_url = "https://www.sebi.gov.in"
                        # Ensure URL is absolute by joining it with the base URL
                        joined_url = urljoin(base_url, url)
                        url = joined_url
                    # Assign the URL to the dictionary key
                    result[key] = url
            return result

        
        def download_menus_js(self):
            """
            Downloads SEBI menus JavaScript file and extracts its content.

            This method fetches the JavaScript file containing SEBI website menus
            from the URL 'https://www.sebi.gov.in/js/menu.js'. It then processes
            the JavaScript content to extract the menu structure by removing 
            JavaScript-specific syntax. The resulting HTML content is wrapped in 
            <html> tags to ensure proper structure.

            Returns:
                str: The HTML content of the SEBI menus extracted from the JavaScript file.
            """
            
            # Define the URL of the JavaScript file containing SEBI menus
            menu_js_url = "https://www.sebi.gov.in/js/menu.js"

            # Fetch the content of the JavaScript file
            response = requests.get(menu_js_url)
            html_content = response.text

            # Process JavaScript content to extract menu structure
            html_content = html_content.replace('document.write("', "")
            html_content = html_content.replace('");', '')
            html_content = "<html>" + html_content + "</html>"

            return html_content

    
        def collect_menu_links(self):
            
            '''
            Collects menu links from the SEBI website and stores them in a CSV file.

            This method retrieves the menu structure from the SEBI website, parses it,
            and organizes it into a DataFrame. It then normalizes and prepares the data
            for storage in a CSV file. The function also ensures that historical data 
            links are included and properly formatted. Finally, it appends the data to 
            an existing CSV file or creates a new one if it doesn't exist.

            '''
            
            
            # OLD => saving menu of sebis into csv file
            # NEW => We will maintain CSV for Menu collection
            if(os.path.exists(urls_of_sebi_menu_csv_path)):
                return
            # OLD => manually downladed fileissues? 
            # NEW => instead of downloaded file, download the file from the server
            
            # OLD => reads a manually saved menus_of_sebi.html file and stores content into html_content
            # NEW => write a method that will get the data using requests.get and store into
            html_content = self.download_menus_js()
            
            # Parse the HTML
            soup = BeautifulSoup(html_content, "html.parser")

            # Get the main list
            ul = soup.ul
            result = self.dictify(ul)

            # Example data dictionary
            data_dict = result

            # Initialize empty lists for each column
            menu_list, submenu_list, description_list, url_list = [], [], [], []

            # Iterate over the data dictionary and append values to the lists
            for menu, submenu_dict in data_dict.items():
                for submenu, details in submenu_dict.items():
                    if isinstance(details, dict):
                        for description, url in details.items():
                            menu_list.append(menu.lower())
                            submenu_list.append(submenu.lower())
                            description_list.append(description.lower())
                            url_list.append(url)
                    else:
                        menu_list.append(menu.lower())
                        submenu_list.append(submenu.lower())
                        description_list.append("")
                        url_list.append(details)

            # Create DataFrame from lists
            df = pd.DataFrame({
                'menu': menu_list,
                'submenu': submenu_list,
                'subsubmenu': description_list,
                'url': url_list
            })
            
            row_count = len(df)
            for i in range(row_count):
                row = df.iloc[i]
                menu = row['menu']
                submenu = row['submenu']
                subsubmenu = row['subsubmenu']
                print(menu, submenu, subsubmenu)
                df.at[i, 'menu'] = menu.replace(" ","_")
                df.at[i, 'submenu'] = submenu.replace(" ","_")
                df.at[i, 'subsubmenu'] = subsubmenu.replace(" ","_")
            print("replaced <space> with <_>")

            '''df2 maintains historical_data links'''
            df2 = pd.DataFrame(
                {
                    'menu': ["legal"] * len(legal_menu_and_sub_menu),
                    'submenu': list(legal_menu_and_sub_menu.keys()),
                    'subsubmenu': ["historical_data"] * len(legal_menu_and_sub_menu),
                    'url': list(legal_menu_and_sub_menu.values())
                }
            )

            final_df = pd.concat([df, df2], axis=0)
            
            # OLD => saved the data to csv file
            # NEW => No change its better to save the menus_data in csv file only
            final_df.to_csv(urls_of_sebi_menu_csv_path, mode='a')
            
            
        def create_folder_hierarchy(self):
            '''
            Creates a folder hierarchy based on the SEBI menu structure.

            This method reads the SEBI menu structure from a CSV file, which contains 
            menu and submenu information. It then creates a folder hierarchy based on 
            this structure in the specified base folder path. Each menu and submenu 
            name is normalized by replacing spaces with underscores and converting to 
            lowercase. If the base folder or any menu/submenu folder doesn't exist, 
            it creates them.
            '''
            try:
                df = pd.read_csv(urls_of_sebi_menu_csv_path)

                # Base folder to store SEBI Extracted Data
                print("Creating Folder Hierarchy for : ",base_folder_path)
                if not os.path.exists(base_folder_path):
                    # Create the base folder if it doesn't exist
                    os.makedirs(base_folder_path)

                for _,row in df.iterrows():
                    menu = row['menu']
                    menu = menu.replace(" ","_")
                    menu = menu.lower()
                    print(menu)
                    menu_folder = os.path.join(base_folder_path,menu)
                    # Create the base folder if it doesn't exist
                    if not os.path.exists(menu_folder):
                        os.makedirs(menu_folder)
                    sub_menu = row['submenu']
                    sub_menu = sub_menu.lower()
                    sub_menu = sub_menu.replace(" ","_")
                    sub_menu_folder = os.path.join(menu_folder,sub_menu)
                    if not os.path.exists(sub_menu_folder):
                        os.makedirs(sub_menu_folder)
                print("Folder Hierarchy Created")
            except Exception as e:
                print(f"Exception occurred : {e}")                

# SEBI DATA SCRAPER
<blockquote>
class SEBIDataScraper:
    '''
    SEBIDataScraper is a class designed for scraping data from the Securities and Exchange Board of India (SEBI) website. 
    It provides functionality to navigate through SEBI's web pages, collect links to HTML and PDF files, and download 
    these files for further analysis or processing.

    Attributes:
        data: A list to store the scraped data.
    
    Methods:
        __init__: Initializes the SEBIDataScraper object and creates a CSV file if it doesn't already exist to store
            links to PDF files.
        navigate_pagination_and_collect_links: Navigates through paginated web pages, extracts links to HTML and PDF 
            files from tables, and stores them in the data attribute.
        collect_html_links: Collects HTML links for a given menu and submenu combination.
        collect_pdf_links_for_all_for_all_for_all: Collects PDF links for a given menu and submenu combination, also extracts PDF text if available.
        download_pdf: Downloads a PDF file from a given URL to the specified download path.
        download_html: Downloads an HTML file from a given URL to the specified download path.
        count_files: Counts the number of files of a specific type (PDF or HTML) in a given menu and submenu folder.
        create_list_of_links: Creates a list of PDF and HTML links for a given menu and submenu.
        download_files: Downloads PDF and HTML files concurrently for a given menu and submenu.
    '''
</blockquote>

In [ ]:
#|exports
#|eval:True 

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import logging
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import concurrent.futures
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import hashlib
import os
import base64
from bs4 import BeautifulSoup

import requests
from requests.adapters import HTTPAdapter
           
class SEBIDataScraper:
    logger_data_scraper = setup_logger("sebi_data_scraper.log")
    def __init__(self):
        self.db_handler_obj = DBHandler()
        
        # OLD => Create CSV files with menus, if they dont already exists
        # NEW => Create sqlitedb (if it doesn't exists) to store the the extracted text 
        # if(not os.path.exists(pdf_links_of_all)):
        #     df = pd.DataFrame(columns=columns_for_pdf_links_of_all)
        #     df.to_csv(pdf_links_of_all, index=True)
        #     print(f"CSV file {pdf_links_of_all} with column names created successfully.")
        self.data = []
        print("SEBI Data Scrapper Object is created")
    
    def navigate_pagination_and_collect_links(self,url,type,sub_type):
        
        # logging.basicConfig(filename='selenium_next_button.log', level=logging.INFO, format='%(asctime)s - %(levelname)s: %(message)s')
        # logging.info(f'Entering navigate_pagination_and_collect_links method, {url}')
        type = type.lower()
        sub_type = sub_type.lower()
        
        options = webdriver.ChromeOptions()
        options.add_argument("--headless=new")
        driver = webdriver.Chrome(options=options)
        driver.get(url)
        page_num = 0

        while(1):
           
            print("inside while")
            try:

                time.sleep(1)
                ########### GEL HTML/PDF LINKS FROM THE TABLE ###########
                html_content = driver.page_source
                print(f"html_content of {page_num} received ")
                soup = BeautifulSoup(html_content,'html.parser')
                table = soup.find('table')
                if(table == None):
                    return
                no_of_rows = len(table.find_all('tr'))
                print(f"Page : {page_num} | No of rows: {no_of_rows}")

                # The html contains table which hold info - PDF Title, Data, PDF Viewer Link
                for row in table.find_all('tr'):
                    if(row == None):
                        continue
                    all_cells = row.find_all('td')
                    #Exclude the first row
                    if(len(all_cells) == 0):
                        continue

                    anchor_tag = all_cells[1].find('a')
                    date = all_cells[0].text
                    title = all_cells[1].text
                    href_link = anchor_tag.get('href')
                    new_row = {
                        "title" : title,
                        "date" : date,
                        "html_link" : href_link,
                        "pdf_link" : "",
                        "type": type,
                        "sub_type" : sub_type,
                        "file_name" : "",
                        "file_type" : "",
                        "pdf_text" : ""
                    }
                    
                    self.data.append(new_row)
                    print("New Row appended to data :", new_row)

                ############ ENDS HERE ############

                time.sleep(2)
                try:
                    # Check if the page contains pagination or is it a single page
                    if not (driver.find_element(By.CLASS_NAME, "pagination_outer")):
                        return

                    WebDriverWait(driver, 20).until(
                        EC.element_to_be_clickable((By.CLASS_NAME, "pagination_outer"))
                        # Check if the pagination_outer is loaded, so we can move on to next step
                    )

                    # if we cant find the next_button, it means we are at the last page
                    if not (driver.find_elements(By.XPATH, "//*[@title='Next']")):
                        return

                    next_button = driver.find_element(By.XPATH, "//*[@title='Next']")
                    time.sleep(2)

                    next_button.click()
                    # logging.info('Clicked on Next Button')
                except NoSuchElementException as e:
                    print(e)
                    print(".pagination_outer is not present and/or next button not present")
                    return

                driver.implicitly_wait(5)
                # logging.info('Waited for download to complete')
            except Exception as e:
                print("EXCEPTION occurred :",e)
                # logging.error(f'An exception occurred: {str(e)}')
                return
            finally:
                #driver.quit()
                page_num += 1
                print("Exiting navigation_pagination_and_collect_links")
                # logging.info('Exiting navigation_pagination_and_collect_links')

    # Function that collects html links from navigating to each menu links
    def collect_html_links(self,menu_to_scrape, submenu_to_scrape):
        print(f"Entering collect_html_links for [{menu_to_scrape}] | [{submenu_to_scrape}]")
        df = pd.read_csv(urls_of_sebi_menu_csv_path)
        
        for _,row in df.iterrows():
            menu = row['menu']
            submenu = row['submenu']
            url = row['url']
            # print(f"Creating list of urls using menu: {menu} and sub_menu: {submenu}")
            menu = menu.replace(" ","_")
            submenu = submenu.replace(" ","_")
            if(menu.lower() == menu_to_scrape.lower() and submenu.lower() == submenu_to_scrape.lower()):
                print(f"Accessing the link {url}")
                self.navigate_pagination_and_collect_links(url,menu,submenu)
            # print(f"Row added in CSV: {menu} | sub_menu: {submenu}")
            
            
    def soup_returner(self,url):
        soup = BeautifulSoup()
        try:
            session = requests.Session()
            retry = HTTPAdapter(max_retries=5)
            session.mount("http://", retry)
            session.mount("https://", retry)
            read = session.get(url,verify=False)
            html_content = read.text
            print(html_content)
            soup = BeautifulSoup(html_content,'html.parser')
            
        except Exception as e:
            print("URL ", url)
            print("Exception occured : ",e )
            
        print("soup returned.")
        return soup
    
    def collect_pdf_link(self,html_link):
        url = html_link
        print(f"Collecting pdf link for {url}")
        soup = self.soup_returner(url)
        only_anchor_tags = soup.find_all('iframe')
        print("only_anchor_tags : ", only_anchor_tags)
        new_pdf_link = ""
        for link in only_anchor_tags:
            print("inside for loop")
            href_link = link.get('src')
            print("href link has been fetched : ",href_link)
            if (href_link!=None) and href_link.lower().endswith(".pdf"):
                pdf_link = href_link
                new_pdf_link = urljoin(home_page_url,pdf_link)
        return new_pdf_link
        
    
    def collect_pdf_links_for_all(self,menu_to_scrape, submenu_to_scrape):
        print("inside collect_pdf_links_for_all")
        self.collect_html_links(menu_to_scrape, submenu_to_scrape)
        print(f"collecting pdf links for : [{menu_to_scrape}] | [{submenu_to_scrape}]")

        for row in self.data:
            url = row['html_link']
            new_pdf_link = self.collect_pdf_link(url)
            # print(f"Collecting pdf link for {url}")
            # soup = self.soup_returner(url)
            # only_anchor_tags = soup.find_all('iframe')
            # new_pdf_link = ""
            # for link in only_anchor_tags:
            #     href_link = link.get('src')
            #     if href_link!=None and href_link.lower().endswith(".pdf"):
            #         pdf_link = href_link
            #         new_pdf_link = urljoin(home_page_url,pdf_link)
            row['pdf_link'] = new_pdf_link

            #storing the name of the pdf for the future use
            split_pdf_link = new_pdf_link.split("/")
            row['file_name'] = split_pdf_link[-1]
            row['file_type'] = "pdf"
            
            print("Pdf link fetched : ", new_pdf_link)
            # If the pdf name is blank that means the content is html file
            
            if (new_pdf_link == ""):
                url_base64 = base64.b64encode(url.encode()).decode()
                # row['file_name'] = url_base64+".html"
                filename_hashed = hashlib.sha256(url_base64.encode('utf-8')).hexdigest()
                row['file_name'] = filename_hashed+".html"
                row['file_type'] = "html"

        # Saving the data into sqlite db
        for row in self.data:
            print(f"saving row in db : {row}")
            self.db_handler_obj.insert_data(row)
        # df2 = pd.DataFrame(self.data)
        # df2.to_csv(pdf_links_of_all, mode='a', index=True)


    def download_pdf(self,url,download_path,file_name):
        logging.basicConfig(filename='selenium_log.log', level=logging.INFO, format='%(asctime)s - %(levelname)s: %(message)s')
        logging.info(f"Downloading {file_name} in {download_path}")
        print(f"Downloading {file_name} in {download_path}")

        if(not url.startswith("http")):
            logging.info("Not a valid url")
            return
        
        pdf_path = os.path.join(download_path,file_name)
        if(os.path.exists(pdf_path)):
            print(f"The pdf {file_name} already exists")
            logging.info(f"The pdf {file_name} already exists")
            return

        try:
            response = requests.get(url)
            logging.info(f'Status of url : {url} is {response.status_code}')
            if(response.status_code != 200):
                print(f"Status of {url} is {response.status_code}")
            options = webdriver.ChromeOptions()

            prefs = {
                "download.default_directory": download_path,
                'download.prompt_for_download': False,
                'plugins.always_open_pdf_externally': True
            }

            options.add_argument("--headless=new")

            options.add_experimental_option('prefs',prefs)

            driver = webdriver.Chrome(options=options)
            driver.get(url)
            logging.info(f'Navigated to URL: {url}')
            # Wait for some time to ensure the PDF is loaded
            #driver.implicitly_wait(10)

            WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.ID, "download"))
            )

            #Find the DOWNLOAD Button from the pdfviewer using the id of button tag
            download_button = driver.find_element(By.ID, "download")
            time.sleep(3)
            download_button.click()
            logging.info('Clicked on PDF download Button')
            driver.implicitly_wait(10)
            logging.info('Waited for download to complete')
            time.sleep(6)

            # num = rename_most_recent_pdf(download_path,file_name,past_num_of_files_present)
            # past_num_of_files_present = num
            # print(f"Past num of files present : {past_num_of_files_present}")

        except Exception as e:
            print("EXCEPTION ",e)
            logging.error(f'An error occurred: {str(e)}')
        finally:
            #driver.quit()
            logging.info('Browser session closed')
    
    def download_pdf_new(self,row):
        url = row['pdf_link']
        html_url = row['html_link']
        title = row['title']
        type = row['type']
        sub_type = row['sub_type']
        file_name = row['file_name']
        
        type = type.replace(" ","_")
        sub_type = sub_type.replace(" ","_")
        # SEBI_data_extraction_base_folder = r"D:\Educational\Sarvam AI\Python\SEBI\SEBI_Extracted_Data"
        type_folder_path = os.path.join(SEBI_data_extraction_base_folder,type)
        sub_type_folder_path = os.path.join(type_folder_path,sub_type)
        download_path = sub_type_folder_path
        
        print("URL Accessing ",url)
        
        file_path = os.path.join(download_path, file_name)
        if(os.path.exists(file_path)):
            print(f"the file {file_path} already exists.")
            return
        
        try:
            response = requests.get(url)
            # logging.info(f'Status of url : {url} is {response.status_code}')
            if(response.status_code != 200):
                print(f"Status of {url} is {response.status_code}")
            options = webdriver.ChromeOptions()

            prefs = {
                "download.default_directory": download_path,
                'download.prompt_for_download': False,
                'plugins.always_open_pdf_externally': True
            }

            options.add_argument("--headless=new")

            options.add_experimental_option('prefs',prefs)

            driver = webdriver.Chrome(options=options)
            driver.get(url)
 

            WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.ID, "download"))
            )

            #Find the DOWNLOAD Button from the pdfviewer using the id of button tag
            download_button = driver.find_element(By.ID, "download")
            time.sleep(2)
            download_button.click()
            
            driver.implicitly_wait(10)
            # logging.info('Waited for download to complete')
            time.sleep(6)
            
            timeout = 25  # Maximum wait time in seconds
            start_time = time.time()
            while time.time() - start_time < timeout:
                print(str(time.time() - start_time))
                if os.path.isfile(os.path.join(download_path, file_name)):
                    print(f"Download complete! for {file_name} in {download_path}")
                    break
                time.sleep(1)  # Check every 1 second
        except NoSuchElementException:
            print(f"Download button not found for URL: {url}")
        finally:
            # driver.quit()
            print("Exiting download_pdf_new")
    
    # @staticmethod
    def download_html(self,url,download_path, file_name):
        print(f"Downloading {file_name} in {download_path}")
        response = requests.get(url)
        
        # filename_hashed = hashlib.sha256(url.encode('utf-8')).hexdigest()

        file_download_path = os.path.join(download_path,file_name)
        if(os.path.exists(file_download_path)):
            print(f"The file {file_name} already exists")
            logging.info(f"The file {file_name} already exists")
            return

        if response.status_code == 200:
            print(response.status_code)
            with open(file_download_path, "w", encoding="utf-8") as f:
                f.write(response.text)
            print(f"{url} is downloaded into {download_path}")
        else:
            print("Failed to download HTML:", response.status_code)
            
    def download_html_new(self,row):
        # url = row['pdf_link']
        html_url = row['html_link']
        title = row['title']
        type = row['type']
        sub_type = row['sub_type']
        file_name = row['file_name']
        
        
        
        print(f" File : {file_name} and LENGTH : {len(file_name)}",file_name)
        
        
        type = type.replace(" ","_")
        sub_type = sub_type.replace(" ","_")
        type_folder_path = os.path.join(SEBI_data_extraction_base_folder,type)
        sub_type_folder_path = os.path.join(type_folder_path,sub_type)
        download_path = sub_type_folder_path
        
        file_path = os.path.join(download_path, file_name)
        if(os.path.exists(file_path)):
            print(f"the file {file_path} already exists.")
            return
        
        response = requests.get(html_url)

        if response.status_code == 200:
            print(response.status_code)
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(response.text)
            print(f"{html_url} is downloaded into {download_path}")
        else:
            print("Failed to download HTML:", response.status_code)
    
    
    # def download_all_files(self):
    #     print("Downloading in...",SEBI_data_extraction_base_folder)
    #     for row in self.data:

    #         pdf_url = row['pdf_link']
    #         html_url = row['html_link']
    #         title = row['title']
    #         sub_type = row['type']
    #         sub_sub_type = row['sub_type']
    #         file_name = row['file_name']

    #         sub_type = sub_type.replace(" ","_")
    #         sub_sub_type = sub_sub_type.replace(" ","_")
            
    #         sub_type_folder_path = os.path.join(SEBI_data_extraction_base_folder,sub_type)
    #         sub_sub_type_folder_path = os.path.join(sub_type_folder_path,sub_sub_type)
    #         download_path = sub_sub_type_folder_path

    #         if (pdf_url == ""):
    #             self.download_html(html_url, download_path, file_name)
    #             continue
    #         self.download_pdf(pdf_url, download_path,file_name)
    
    def count_files(self, menu,submenu,file_type):
    # Ensure the folder exists

        folder_path = os.path.join(SEBI_data_extraction_base_folder,menu)
        folder_path = os.path.join(folder_path,submenu)
        if not os.path.exists(folder_path):
            print(f"The folder '{folder_path}' does not exist.")
            return

        # Initialize the count
        file_count = 0
        total_files = 0
        # Iterate through all items in the folder
        for item in os.listdir(folder_path):
            # Check if the item is a file
            if os.path.isfile(os.path.join(folder_path, item)):
                # If it's a file, increment the count
                total_files += 1
                if(item.lower().endswith(file_type)):
                    file_count += 1

        return file_count,total_files
    
    def create_list_of_links(self,menu,submenu):

        df = pd.read_csv(pdf_links_of_all)
        pdf_link_list = []
        html_link_list = []
        
        print(f"Create list of links {menu} and {submenu}")
                
        # Fetch the rows from the table having type==menu and sub_type==submenu
        fetched_result = []
        # What will be returned here?
        fetched_result = self.db_handler_obj.fetch_rows_by_type_and_subtype(menu.lower(),submenu.lower())
        
        for row in fetched_result:
            print("Fetched Result Row:")
            print(row)
            html_link,title,date,pdf_link,type,sub_type,file_name,file_type,pdf_text = row
            
            def create_download_path(type, sub_type):
                type = type.replace(" ","_")
                sub_type = sub_type.replace(" ","_")
                
                type_folder_path = os.path.join(SEBI_data_extraction_base_folder,type)
                sub_type_folder_path = os.path.join(type_folder_path,sub_type)
                download_path = sub_type_folder_path
                return download_path
            
            download_path = create_download_path(type,sub_type)
            file_path = os.path.join(download_path,file_name)
            
            if(pdf_link == "" or pdf_link == None):
                # It means the file is html (NOT PDF)
                if(os.path.exists(file_path)):
                    print(f"Not adding to list, becoz The file {file_path} already exists")
                    continue
                pdf_link_list.append(row)
            else:
                if(os.path.exists(file_path)):
                    print(f"Not adding to list, becoz The file {file_path} already exists")
                    continue
                html_link_list.append(row)
                
        return pdf_link_list,html_link_list

    
    # def download_files(self, menu, sub_menu):
    #     df = pd.read_csv(pdf_links_of_all)
        
    #     for _,row in df.iterrows():
    #         pdf_url = row['pdf_link']
    #         html_url = row['html_link']
    #         title = row['title']
    #         type = row['type']
    #         sub_type = row['sub_type']
    #         file_name = row['file_name']
    #         flag = False
    #         if(pd.isna(row['pdf_link'])):
    #             flag = True
    #         if(pd.isna(html_url) or pd.isna(title) or pd.isna(type) or pd.isna(sub_type) or pd.isna(file_name)):
    #             print("Values are empty.")
    #             continue
            
    #         type = type.replace(" ","_")
    #         sub_type = sub_type.replace(" ","_")
            
    #         type_folder_path = os.path.join(SEBI_data_extraction_base_folder,type)
    #         sub_type_folder_path = os.path.join(type_folder_path,sub_type)
    #         download_path = sub_type_folder_path
            
    #         if (sub_menu == None):
    #             if(menu == type.lower()):
    #                 if (flag):
    #                     file_name = row['file_name']
    #                     filename_hashed = hashlib.sha256(html_url.encode('utf-8')).hexdigest()
    #                     row['file_name'] = filename_hashed
    #                     self.download_html(html_url, download_path, filename_hashed)
    #                 else:
    #                     self.download_pdf(pdf_url, download_path,file_name)
    #         else:
    #             if(menu == type.lower() and sub_menu == sub_type.lower()):
    #                 if (flag):
    #                     file_name = row['file_name']
    #                     filename_hashed = hashlib.sha256(html_url.encode('utf-8')).hexdigest()
    #                     row['file_name'] = filename_hashed
    #                     self.download_html(html_url, download_path, filename_hashed)
    #                 else:
    #                     self.download_pdf(pdf_url, download_path,file_name)
                        
    # def download_files2(self, row, menu, sub_menu):
        
    #     pdf_url = row['pdf_link']
    #     html_url = row['html_link']
    #     title = row['title']
    #     type = row['type']
    #     sub_type = row['sub_type']
    #     file_name = row['file_name']
        
    #     print(f"Downloading..... {html_url}")
        
    #     type = type.replace(" ","_")
    #     sub_type = sub_type.replace(" ","_")
        
    #     type_folder_path = os.path.join(SEBI_data_extraction_base_folder, type)
    #     sub_type_folder_path = os.path.join(type_folder_path, sub_type)
    #     download_path = sub_type_folder_path

    #     if not os.path.exists(download_path):
    #         os.makedirs(download_path)
            
    #     if (sub_menu == None):
    #         if(menu == type.lower()):
    #             if (pdf_url == ""):
    #                 self.download_html(html_url, download_path, file_name)
    #             else:
    #                 self.download_pdf(pdf_url, download_path,file_name)
    #         else:
    #             if(menu == type.lower() and sub_menu == sub_type.lower()):
    #                 if (pdf_url == ""):
    #                     self.download_html(html_url, download_path, file_name)
    #                 else:
    #                     self.download_pdf(pdf_url, download_path,file_name)
                        
    def download_files3(self,menu,submenu):
        print("inside download_files3")
        print(menu,submenu)
        pdf_urls = []
        html_urls = []
        pdf_urls,html_urls = self.create_list_of_links(menu,submenu)
        # pdf_urls,html_urls,total_pdf_count,total_html_count = self.create_list_of_links(menu,submenu)
        for row in pdf_urls:
            print(row)
            print(row['pdf_link'])
            print("-------------------")
        print("HTML Links ::: ")
        for row in html_urls:
            print(row['html_link'])
            print("-------------------") 
        
        print("List has been created, moving on to downloads")
        
        # Concurrently download PDFs
        round = 1
        file_count,total_files_downloaded = self.count_files(menu,submenu,"pdf")
        i = 0
        
        # for row in pdf_urls:
        #     self.download_pdf_new(row)
        #     print(f"Download completed : {i}/{len(pdf_urls)} ")
        #     i += 1
        while(1):
            with concurrent.futures.ThreadPoolExecutor() as executor:
                executor.map(self.download_pdf_new, pdf_urls)
            pdf_urls = []
            html_urls = []
            pdf_urls,html_urls,total_pdf_count,total_html_count = self.create_list_of_links(menu,submenu)
            
        ##########################      
        # while(total_files_downloaded < total_pdf_count):
        #     print(f"Round : {round}")
        #     with concurrent.futures.ThreadPoolExecutor() as executor:
        #         executor.map(self.download_pdf_new, pdf_urls)
        #     # for row in pdf_urls:
        #     #     self.download_pdf_new(row)
        #     file_count,_ = self.count_files(menu,submenu,"pdf")
        #     print(f"File Count : {file_count}/{total_files_downloaded}")
        #     round += 1
        ####################
        
        # Concurrently download HTMLs
        round = 1
        i = 0
        file_count,total_files_downloaded = self.count_files(menu,submenu,"html")
        for row in html_urls:
            self.download_html_new(row)
            print(f"Download completed : {i}/{len(html_urls)} ")
            i += 1
        # while(total_files_downloaded < total_html_count):
        #     print(f"Round : {round}")
        #     # with concurrent.futures.ThreadPoolExecutor() as executor:
        #     #     executor.map(self.download_html_new, html_urls)
        #     for row in html_urls:
        #         self.download_html_new(row)
        #     file_count,_ = self.count_files(menu,submenu,"html")
        #     print(f"File Count : {file_count}/{total_files_downloaded}")
        #     round += 1
        ###########################
        
        
        
        
        
    # def download_files_concurrently(self, menu, sub_menu):
    #     pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())  # Use number of available CPUs
    #     func = lambda row: self.download_file(row, menu, sub_menu)
    #     df = pd.read_csv(pdf_links_of_all)
    #     pool.map(func, df.iterrows())
    #     pool.close()
    #     pool.join()
  

# DBHandler - Methods to handle Database

In [ ]:
#|exports
#|eval:True
import sqlite3

class DBHandler():
    
    def __init__(self):
        self.create_db()
        self.create_tables()
        pass
    
    def create_db(self):
        if not os.path.exists(DB_NAME):
            print("Database does not exist. Creating it....")
            open(DB_NAME, 'w').close()
            # Create Tables under it  
            self.create_tables()
        else:
            print(f"database {DB_NAME} already exists.")
            
    def create_tables(self):
        print(f"Creating table [{TABLE_SEBI_RECORDS}] in database :[{DB_NAME}]")
        
        try:
            conn = sqlite3.connect(DB_NAME)
            cursor = conn.cursor()

            query1 = f'''CREATE TABLE IF NOT EXISTS {TABLE_SEBI_RECORDS} (
                            html_link TEXT PRIMARY KEY,
                            title TEXT,
                            date TEXT,
                            pdf_link TEXT,
                            type TEXT,
                            sub_type TEXT,
                            file_name TEXT,
                            file_type TEXT,
                            pdf_text TEXT
                        )'''
            
            cursor.execute(query1)
            
            conn.commit()
            print("tables successfully created!")
        except Exception as e:
            print("Exception occured while creating table : ",e)
        finally:
            if conn:
                conn.close()
                
    def insert_data(self,row):
        print(f"Inserting values into {DB_NAME} and table {TABLE_SEBI_RECORDS}")

        html_link = row['html_link']
        title = row['title']
        date = row['date']
        pdf_link = row['pdf_link']
        type = row['type']
        sub_type = row['sub_type']
        file_name = row['file_name']
        file_type = row['file_type']
        pdf_text = row['pdf_text']
        
        try:
            conn = sqlite3.connect(DB_NAME)
            cursor = conn.cursor()

            # Insert data into the database
            query = f'''INSERT INTO TABLE_SEBI_RECORDS (html_link, title, date, pdf_link, 
            type, sub_type, file_name, file_type, pdf_text) 
            VALUES (?,?,?,?,?,?,?,?,?)'''

            cursor.execute(query, (html_link,title,date,pdf_link,type,sub_type,file_name,file_type,pdf_text))

            # Commit changes and close connection
            conn.commit()
            print(f"Data Scored Successfully in {TABLE_SEBI_RECORDS}")
        except Exception as e:
            print(f"Exception occured while inserting a row : {row}")
            print(e)
        finally:
            if conn:
                conn.close()
                
    def update_data(row, pdf_text):
        html_link = row['html_link']
        print(f"updating a pdf_text for {html_link}")
        try:            
            conn = sqlite3.connect(DB_NAME)
            cursor = conn.cursor()

            # Update the pdf_text for the given html_link
            cursor.execute("UPDATE TABLE_SEBI_RECORDS SET pdf_text = ? WHERE html_link = ?", (pdf_text, html_link))
            
            # Commit the transaction
            conn.commit()
            print("PDF text updated successfully.")
        except sqlite3.Error as e:
            print(f"Exception occured for {html_link}: ", e)
        finally:
            # Close the database connection
            if conn:
                conn.close()
                
                
    def fetch_rows_by_type_and_subtype(type,sub_type):
        try:
            # Connect to the SQLite database
            conn = sqlite3.connect(DB_NAME)
            cursor = conn.cursor()

            # Execute the SQL query to fetch rows based on the condition
            cursor.execute(f"SELECT * FROM {TABLE_SEBI_RECORDS} WHERE type = ? AND sub_type = ?", (type,sub_type))
            
            # Fetch all rows that satisfy the condition
            rows = cursor.fetchall()
            return rows

        except sqlite3.Error as e:
            print("Error fetching rows:", e)
            return None
        finally:
            # Close the database connection
            if conn:
                conn.close()

In [ ]:
#|include: True
#|eval: True
menu_collectors = SEBIMenuCollector()
scraper = SEBIDataScraper()
menu_collectors.collect_menu_links()
menu_collectors.create_folder_hierarchy()

In [ ]:
#|include: False
#|eval: False
#|hide
new_pdf_link = scraper.collect_pdf_link("https://www.sebi.gov.in/legal/rules/mar-2021/sebi-annual-report-rules-2021_49611.html")
print(new_pdf_link)

In [ ]:
#|include: False
#|eval: False
#|hide
scraper.collect_pdf_links_for_all("legal", "rules")

In [ ]:
class DataExtractor:
    def __init__(self) -> None:
        pass
    
    # Traverse a sql table row by row and whenever pdf_text is NULL, go to the download directory => check if the file present, if yes => extract the pdf_text using ocr
    # The below function will just return the pdf_text by accessing the file_path
    # check if the file_path exists or not
    def extract_data_from_pdf(file_path):
        pdf_text = ""
        if(not os.path.exists(file_path)):
            print(f"The {file_path} doesnt exists!")
            return pdf_text
        
        